In [1]:
#Want to find the Min Temperature per day

import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')

from pyspark import SparkContext, SparkConf
conf = SparkConf().setMaster('local').setAppName('weather1800')
sc = SparkContext(conf = conf)

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('weather1800').getOrCreate()

# Import Data

In [2]:
folder = '/home/ubuntu/data/raw'
file = '/taming-BD-1800.csv'

#lines = spark.read.csv(folder+file, header=True, inferSchema=True)
#lines.printSchema()

lines_rdd = sc.textFile(folder+file)

In [3]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3])*0.1*(9.0 / 5.0) + 32.0
    return(stationID, entryType, temperature)

In [4]:
parsedLines = lines_rdd.map(parseLine)
parsedLines.collect()[0:10]

[('ITE00100554', 'TMAX', 18.5),
 ('ITE00100554', 'TMIN', 5.359999999999999),
 ('GM000010962', 'PRCP', 32.0),
 ('EZE00100082', 'TMAX', 16.52),
 ('EZE00100082', 'TMIN', 7.699999999999999),
 ('ITE00100554', 'TMAX', 21.2),
 ('ITE00100554', 'TMIN', 9.5),
 ('GM000010962', 'PRCP', 32.0),
 ('EZE00100082', 'TMAX', 24.08),
 ('EZE00100082', 'TMIN', 8.599999999999998)]

In [5]:
tMin_rdd = parsedLines.filter(lambda x: 'TMIN' in x[1])
prcp_rdd = parsedLines.filter(lambda x: 'PRCP' in x[1])
prcp

NameError: name 'prcp' is not defined

In [ ]:
#After exploration, data is: 
    #temperature data per day in Italy and Czech Republic
    #percipitionat per day in Paris

# Plot Min Temp By City Per Day

In [ ]:
#Print out temps of each city
import pandas as pd

#How many Station ids are there?
weather_pd = pd.DataFrame(parsedLines.collect(),columns=['station','entryType','temperature'])
weather_pd.head()
stations = weather_pd.groupby('station').mean()
#print(stations.head())

tMin = parsedLines.filter(lambda x: 'TMIN' in x[1])

Czech_tMin = tMin_rdd.filter(lambda x: 'EZE00100082' in x[0]).collect()
Italy_tMin = tMin_rdd.filter(lambda x: 'ITE00100554' in x[0]).collect()
Paris_tMin = tMin_rdd.filter(lambda x: 'GM000010962' in x[0]).collect()

Czech_tMin_temp = []
for row in Czech_tMin:
    Czech_tMin_temp.append(row[2]) 
Italy_tMin_temp = []
for row in Italy_tMin:
    Italy_tMin_temp.append(row[2]) 

    
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
sns.set(style="white", palette="muted", color_codes=True)

sns.tsplot(Czech_tMin_temp, color='c')
sns.tsplot(Italy_tMin_temp, color='m')
plt.legend()
plt.title('Min Temp (F) by Day Year =1800')
plt.show()

# Find Min Temp Per Year

In [ ]:
stationTemps = tMin_rdd.map(lambda x: [x[0], x[2]])
stationTemps.collect()[1]

#Now we have a key value pair.

minTemps_rdd = stationTemps.reduceByKey(lambda x, y: min(x,y))

results = minTemps_rdd.collect()
for result in results:
    print(result)